In [1]:
import os
import argparse
import json
import time

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

from urllib.parse import urlparse


In [2]:
def video_id(url):
    """
    Examples:
    - http://youtu.be/SA2iWivDJiE
    - http://www.youtube.com/watch?v=_oPAwA_Udwc&feature=feedu
    - http://www.youtube.com/embed/SA2iWivDJiE
    - http://www.youtube.com/v/SA2iWivDJiE?version=3&amp;hl=en_US
    """
    o = urlparse(url)
    if o.netloc == 'youtu.be':
        return o.path[1:]
    elif o.netloc in ('www.youtube.com', 'youtube.com'):
        if o.path == '/watch':
            id_index = o.query.index('v=')
            return o.query[id_index+2:id_index+13]
        elif o.path[:7] == '/embed/':
            return o.path.split('/')[2]
        elif o.path[:3] == '/v/':
            return o.path.split('/')[2]
    return None

In [5]:
# Set DEVELOPER_KEY to the API key value from the APIs & auth > Registered apps
# tab of
#   https://cloud.google.com/console
# Please ensure that you have enabled the YouTube Data API for your project.
with open("../private/youtube", "r") as r:
  for line in r:
    DEVELOPER_KEY = line[:-1]
    break
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

In [6]:
def videos_list_multiple_ids(youtube, videoids):
  response = youtube.videos().list(
    
  ).execute()

  return response['items']

In [33]:
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)


links = []
videoIds = []
batchSize = 50
with open("./data_export/url_stats/16-dec/expanded_url_map.json", "r") as r:
  data = json.load(r)

In [34]:
youtube_data = []
for key, vv in data.items():
  v = vv['expanded_url']
  if 'domain' in v:
    if v['domain']['domain'] == 'youtube':
      vid = video_id(key)
      if vid is None:
        continue
      videoIds.append(vid)
      record = {'video_id': vid}
      record.update(vv)
      record['url'] = key
      youtube_data.append(record)

In [46]:
key

'https://saraacarter.com/report-ag-barr-knew-about-two-federal-probes-into-hunter-biden-since-the-spring/'

In [35]:
total = len(videoIds)

In [36]:
cnt = 0
responses = []
while cnt < total:
  batch = videoIds[cnt:cnt+batchSize]
  cnt += batchSize
  try:
    video_response = youtube.videos().list(id=','.join(batch),
                part='snippet').execute()
    for item in video_response['items']:
      responses.append(item)
  except HttpError as e:
    print('An HTTP error {} occurred:\n{}'.format(e.resp.status, e.content))
    if e.resp.status == 403:
      cnt -= batchSize
      time.sleep(3600* 12) # Sleep for one hour if hits rate limit 
      print('Hits rate limit, sleep for one hour.')
  print("{} / {} processed".format(cnt, total))

50 / 358 processed
100 / 358 processed
150 / 358 processed
200 / 358 processed
250 / 358 processed
300 / 358 processed
350 / 358 processed
400 / 358 processed


In [37]:
video_response

{'kind': 'youtube#videoListResponse',
 'etag': 'N3cH4-8Qyt6di3nGkvNQ0-9oBYc',
 'items': [],
 'pageInfo': {'totalResults': 0, 'resultsPerPage': 0}}

In [38]:
responses

[]

In [39]:
batch

['w39mk-wrkhm',
 'wjuz-kfs_qo',
 'x1uyg1f0rna',
 'xqurkiik_3s',
 'yrqi9hcd_ts',
 'yubo2dpjeji',
 'z_fndaqiohy',
 'ztu5y5obwpk']

In [44]:
    video_response = youtube.videos().list(id='W39MK-WRKhM',
                part='snippet').execute()

In [45]:
video_response

{'kind': 'youtube#videoListResponse',
 'etag': 'WgmImiC992x9Fg8eG9Yo4w267aE',
 'items': [{'kind': 'youtube#video',
   'etag': 'MiSNfQbkJFRB0LC8JeaxwdAayVM',
   'id': 'W39MK-WRKhM',
   'snippet': {'publishedAt': '2020-12-14T19:49:23Z',
    'channelId': 'UCHqC-yWZ1kri4YzwRSt6RGQ',
    'title': '🔴 LIVE: BREAKING Nevada GOP ELECTORS CAST VOTES FOR PRESIDENT TRUMP!!',
    'description': 'December 14, 2020: Watch LIVE as the Nevada GOP make a special announcement from Carson City regarding the 2020 election\n\nJoin this channel to get access to perks:\nhttps://www.youtube.com/channel/UCHqC-yWZ1kri4YzwRSt6RGQ/join\n\n🔴  Subscribe to RSBN for more LIVE streams and Breaking NEWS: http://bit.ly/2gDGbkh\n\n🔴  Donate to RSBN and help keep us on the air: http://rsbn.tv/donate\n\n🔴  Follow us on Twitter for breaking news updates: http://twitter.com/RSBNetwork\n\n🔴  Like us on Facebook for more live streams and updates: http://facebook.com/rightsidebroadcasting',
    'thumbnails': {'default': {'url':